In [290]:
class_names = ['생활문화', '스포츠', '정치', '사회', 'IT과학', '경제', '세계']

# 클래스와 인덱스 매핑
class_to_idx = {
    '생활문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    'IT과학': 4,
    '경제': 5,
    '세계': 6
}

In [291]:
import torch
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader

import cleanlab
from cleanlab.filter import find_label_issues

In [292]:
train_df = pd.read_csv('../data/train/woosama_cleanlab.csv')
preliminary_labels = train_df['target']

In [293]:
output_df = pd.read_csv('output/sec_woosama_with_prob.csv')
model_pred_probs = output_df[[f'prob_{i}' for i in range(7)]].values

In [294]:
# 초기 라벨과 모델 예측 확률을 통해 라벨 오류 의심 샘플 찾기
label_issues = find_label_issues(
    labels=preliminary_labels,       # 초기 라벨
    pred_probs=model_pred_probs,     # 모델의 예측 확률
)
label_issues

array([False, False, False, ..., False, False, False])

In [295]:
# from cleanlab.dataset import health_summary
# health_stats = health_summary(
#     labels=preliminary_labels,
#     pred_probs=model_pred_probs,
#     class_names=class_names
# )

In [296]:
# 라벨 오류 가능성이 있는 샘플의 인덱스만 추출
label_issue_indices = np.where(label_issues)[0]

# 오류 가능성이 높은 샘플의 인덱스만 리스트로 가져오기
issue_indices = train_df.index[label_issue_indices]
issue_indices

Index([513, 2239, 5639, 6562, 6785, 8010], dtype='int64')

In [297]:
train_df = train_df.drop(columns=['Unnamed: 0'])

In [298]:
# 오류 샘플에서 모델이 예측한 라벨 보기
corrected_labels = [model_pred_probs[idx].argmax() for idx in label_issue_indices]

In [299]:
# 라벨을 모델의 예측 라벨로 교체
train_df.loc[label_issue_indices, 'target'] = corrected_labels
train_df

,target,text,ID,target_name,re_text
0,3,"정보 공개 중요성, 투명한 정부 운영 필수",NaN,NaN,NaN
1,4,"기술 발전과 인간 가치 중시, 현명한 사회 발전 방향 모색",NaN,NaN,NaN
2,4,"데이터 분석 기반 의사결정 지원 시스템 개발, 효율적인 관리 시스템 구축",NaN,NaN,NaN
3,3,지식 기반 경제 구축을 위한 노력,NaN,NaN,NaN
4,3,"지역 주민들의 의견 수렴, 맞춤형 정책 개발 추진",NaN,NaN,NaN
...,...,...,...,...,...
8412,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,ynat-v1_train_02795,세계,트럼프 폭스뉴스 앵커들 충성도 점수 매겨...10점 만점에 12점도
8413,4,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,ynat-v1_train_02796,정치,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠
8414,4,텔레그램 텔레의 서 시간 다운 종,ynat-v1_train_02797,IT과학,텔레그램 등 아시아서 2시간 다운...카카오뱅크 서비스 장애 39분 지속(2보)
8415,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,ynat-v1_train_02798,스포츠,"인터뷰: 류현진, 친구에게 안타 맞는 것 싫어해...승부는 냉정"


In [288]:
train_df = train_df.drop(columns=['Unnamed: 0.1'])
train_df = train_df.drop(columns=['Unnamed: 0.2'])

In [230]:
# from cleanlab.dataset import health_summary
# health_stats = health_summary(
#     labels=train_df['target'],
#     pred_probs=model_pred_probs,
#     class_names=class_names
# )

In [300]:
train_df.to_csv('../data/train/sec_woosama_cleanlab.csv')

In [301]:
# 각 target별 개수와 비율 구하기
target_counts = train_df['target'].value_counts()
target_ratios = train_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
4        1433  0.170251
1        1294  0.153736
2        1230  0.146133
5        1206  0.143281
0        1149  0.136509
6        1132  0.134490
3         973  0.115599
